In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/php.jsonl

In [2]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
ls = []

with open('php.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

300000it [00:23, 12659.92it/s]


297509

In [4]:
client = InferenceClient(
    "", timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [5]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [6]:
!mkdir mixtral-php
# !rm -rf mixtral-php/*.json

mkdir: cannot create directory ‘mixtral-php’: File exists


In [7]:
def answer(q, i):
    filename = f'mixtral-php/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality PHP problem. Present your output in two distinct sections: [Problem Description] and [Solution].

PHP snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included but do not mention code snippet above.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [10]:
answer(*urls[1])

In [11]:
with open('mixtral-php/1.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

You are working on a PHP project using the Yii2 framework that involves a table called "ref\_scale" in the database. This table has the following columns: type, name, raw, scaled, attribute\_1, attribute\_2, and attribute\_3. Each of these columns is a string with a maximum length of 255 characters.

Your task is to create a function that will calculate the average value of the "scaled" column for each unique value of the "type" column. The function should return an associative array with the unique values of the "type" column as keys and their corresponding average value of the "scaled" column as values.

However, you have noticed that there are some missing or invalid values in the "scaled" column, and you want to exclude these from the calculation. A value is considered valid if it is a non-empty string. You will also need to consider the following edge cases:

* If there are no valid values for a given type, then that type should not be present in the fi

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  6%|█████▋                                                                                          | 17721/297509 [3:47:35<1240:59:53, 15.97s/it]